In [33]:
import pandas
import matplotlib
import requests
import warnings

import sys
sys.path.append('..')

from helper import load_data , load_repo , GIT_API
warnings.filterwarnings('ignore')

%matplotlib inline

import os
import json
import time

In [34]:
json_store_dir = '..'+os.sep+'data'+os.sep+'json'

In [35]:
testrepo = 'https://github.com/rust-lang/cargo'

ISSUE = 'issue'
PULL = 'pull'

time_sleep = 0.1

OPEN = 'open'
CLOSED = 'closed'

In [48]:
def get_pull_pages(repo):
    pr = 'https://api.github.com/repos/{}/{}/pulls?state=all&client_id={}&client_secret={}'.format(
                repo.split('/')[3] ,
                repo.split('/')[4] ,
                GIT_API[0].split(':')[0] , GIT_API[0].split(':')[1])
    req = requests.get(pr)
    return int(req.headers['Link'].split(',')[1].split('page=')[1].split('>')[0])

In [54]:
#this function fetches review comment of pull request and returns a dataframe of review comments
def get_pull_requests(repo,pagenum):
    df = pandas.DataFrame()
    add_address(repo+str(pagenum))
    
    repo_parts = repo.split('/')
    repo_full = repo_parts[-2]+'/'+repo_parts[-1]
    
    par_folder = json_store_dir+os.sep+'pull_request'+os.sep+repo_full.replace('/',os.sep)
    json_store_location = par_folder+os.sep+'pulls'+str(pagenum)+'.json'
        
    if os.path.exists(json_store_location):
        with open(json_store_location) as f:
            pulls = json.load(f)
    else:
        pr = 'https://api.github.com/repos/{}/{}/pulls?state=all&client_id={}&client_secret={}&page={}'.format(
                repo.split('/')[3] ,
                repo.split('/')[4] ,
                GIT_API[6].split(':')[0] , GIT_API[6].split(':')[1],
                pagenum)
            
        time.sleep(time_sleep)
        req = requests.get(pr)
        print(req.headers['X-RateLimit-Remaining'])
        
        pulls = req.json()
        print(pr)
        if not os.path.exists(par_folder):
            os.makedirs(par_folder)
        with open(json_store_location, 'w') as outfile:
            json.dump(pulls, outfile)
        
        
    for pullreq in pulls:
        df = df.append({
            "project_name": repo,
            "pr_id": pullreq["number"],
            "id": pullreq['id'],
            "user_login": pullreq['user']['login'] if pullreq['user'] else 'null',
            "created_at": pullreq['created_at'],
            "updated_at": pullreq['updated_at'],
            "closed_at": pullreq['closed_at'],
            "merged_at": pullreq['merged_at'],
            "merge_commit_sha": pullreq['merge_commit_sha'],
            "author_assoc": pullreq['author_association'],
            "is_open": 0 if pullreq['state'] == 'closed' else 1},ignore_index=True)
    return df 

# Open/Closed Pull Request Comments

In [ ]:
selected_repos = pandas.read_csv('../data/depend/selected_dependencies.csv')

for repo in selected_repos.Repository_URL.unique():
    
    print(repo )
    
    number_of_pages = get_pull_pages(repo)
    for i in range(1,number_of_pages+1):
        pullrequests = pullrequests.append(get_pull_requests(repo,i),ignore_index=True)
        pullrequests.to_csv('../data/pullrequests.csv.gz',compression='gzip')
        
    
    prcoessed_addresses.to_csv('../data/padd.csv')

https://github.com/serde-rs/serde
4922
https://api.github.com/repos/serde-rs/serde/pulls?state=all&client_id=093a7ed958a206f60a37&client_secret=3dc925411cdea302056f7d22a5123efe2dbad079&page=1
4921
https://api.github.com/repos/serde-rs/serde/pulls?state=all&client_id=093a7ed958a206f60a37&client_secret=3dc925411cdea302056f7d22a5123efe2dbad079&page=2
4920
https://api.github.com/repos/serde-rs/serde/pulls?state=all&client_id=093a7ed958a206f60a37&client_secret=3dc925411cdea302056f7d22a5123efe2dbad079&page=3
4919
https://api.github.com/repos/serde-rs/serde/pulls?state=all&client_id=093a7ed958a206f60a37&client_secret=3dc925411cdea302056f7d22a5123efe2dbad079&page=4
4918
https://api.github.com/repos/serde-rs/serde/pulls?state=all&client_id=093a7ed958a206f60a37&client_secret=3dc925411cdea302056f7d22a5123efe2dbad079&page=5
4917
https://api.github.com/repos/serde-rs/serde/pulls?state=all&client_id=093a7ed958a206f60a37&client_secret=3dc925411cdea302056f7d22a5123efe2dbad079&page=6
4916
https://api.g